In [2]:
import matplotlib.pyplot as plt
import pandas as pd

# Read the features dataframe from CSV
playlists_df = pd.read_csv('/content/drive/MyDrive/Spotify_Data/playlist_df.csv')
tracks_df = pd.read_csv('/content/drive/MyDrive/Spotify_Data/final_tracks_data.csv')
ratings_df = pd.read_csv('/content/drive/MyDrive/Spotify_Data/ratings_df.csv')

In [3]:
tracks_df

,artist_name,track_uri,artist_uri,track_name,album_uri,album_name,track_id,genres,artist_popularity,release_date,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,Missy Elliott,0UaMYEvWZi0ZqiDOoHU3YI,2wIVse2owClT7go1WT98tk,Lose Control (feat. Ciara & Fat Man Scoop),6vV5UrXcfyQD1wu4Qo2I9K,The Cookbook,2209,"['dance pop', 'hip hop', 'hip pop', 'neo soul'...",69,2005-07-04,...,-5.370,0.0,0.0308,0.098100,0.019800,0.4340,0.5490,122.973,267355.0,4.0
1,Britney Spears,6I9VzXrHxO9rA9A5euc8Ak,26dSoYclwsYLMAKD3tpOr4,Toxic,0z7pVBGOD7HCIB7S8eLkLI,In The Zone,27709,"['dance pop', 'pop']",78,2003-11-13,...,-8.205,0.0,0.3640,0.075100,0.000000,0.1280,0.5840,175.906,239320.0,4.0
2,Beyoncé,0WqIKmW4BTrj3eJFmnCKMv,6vWDO969PvNqNYHIOW5v0m,Crazy In Love,25hVFAxTlDvXbx2X2QkUkE,Dangerously In Love (Alben für die Ewigkeit),2383,"['pop', 'r&b']",85,2003-06-23,...,-8.811,1.0,0.0453,0.000064,0.832000,0.1170,0.6510,121.035,371572.0,4.0
3,Justin Timberlake,1AWQoqb9bSvzTjaLralEkT,31TPClRtHm23RisEBtV3X7,Rock Your Body,6QPkyl04rXwTGlGlcYaRoW,Justified,5177,"['dance pop', 'pop']",77,2002-11-04,...,-3.175,0.0,0.1300,0.067900,0.094700,0.3910,0.1360,129.095,254853.0,4.0
4,Shaggy,1lzr43nnXAijIGYnCT8M8H,5EvFsr3kj42KNv97ZEnqij,It Wasn't Me,6NmFmPX56pcLBOFMhIiKvF,Hot Shot,7784,"['dance pop', 'pop rap', 'reggae fusion']",71,2000-01-01,...,-2.404,0.0,0.1210,0.001740,0.651000,0.0255,0.3130,176.046,227280.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323353,Olivia Penalva,6sifTSpWzBr2WfXrzX3lnw,6ytGxUYeXamODJwiXuZvjO,Skyline,4QnTenYw3qy3u3qaHyNgXx,Weightless,322564,[],37,2016-08-08,...,-16.224,1.0,0.0339,0.701000,0.841000,0.3010,0.0461,88.320,178800.0,3.0
323354,Dan Talevski,3Jg2JhI8ykraUjO4Qk4LPC,0Vu4A064M7J86SoMA0nrJH,My Religion,7yk6wYsl2WVlBMsz2OLaXa,High Times,319090,['canadian pop'],35,2017-10-27,...,-9.448,1.0,0.1520,0.095100,0.000000,0.1140,0.2960,127.915,240040.0,4.0
323355,Adele,5xIYF3JwqNSqQG2TtlfrYq,4dpARuHxo51G3z768sgnrY,Water Under The Bridge,0vT3XBE3jdXc2uDmutViig,25,321712,"['british soul', 'pop', 'pop soul', 'uk pop']",83,2015-11-19,...,-11.304,1.0,0.0893,0.924000,0.000044,0.0976,0.8600,168.707,132440.0,4.0
323356,Stanaj,23L6jhZ95pj6Wr5HwVBVil,3zrUX1hQrUB9aXcOiyQLmN,Find Me,2ekHnPrPK0Fqxt0mSGDbZs,From A Distance,317864,[],38,2017-01-27,...,-10.207,1.0,0.0284,0.208000,0.000000,0.0920,0.3200,95.275,290893.0,3.0


In [19]:
def remove_unnamed_columns(df):
    return df.loc[:, ~df.columns.str.contains('^Unnamed')]

playlists_df = remove_unnamed_columns(playlists_df)
tracks_df = remove_unnamed_columns(tracks_df)
ratings_df = remove_unnamed_columns(ratings_df)

In [20]:
!pip install implicit

In [22]:
from implicit.als import AlternatingLeastSquares
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from scipy.sparse import coo_matrix

# Load your data
# ...

# Convert implicit feedback data to a binary sparse matrix indicating song presence
interaction_matrix = coo_matrix((np.ones(len(ratings_df)), (ratings_df['pid'], ratings_df['track_id'])))

# Train WRMF model
als_model = AlternatingLeastSquares(factors=50, regularization=0.01, iterations=50)
als_model.fit(interaction_matrix)

# Get user and item factors from WRMF model
user_factors = als_model.user_factors

# Extract additional features from tracks_df, including track_id
additional_features = tracks_df[['track_id', 'danceability', 'energy', 'key', 'loudness', 'mode',
                                 'speechiness', 'acousticness', 'instrumentalness',
                                 'liveness', 'valence', 'tempo']].values

# Create playlist features by averaging additional features for each playlist
unique_playlist_ids = np.unique(ratings_df['pid'])
playlist_features = np.zeros((len(unique_playlist_ids), additional_features.shape[1] - 1))
for index, playlist_id in enumerate(unique_playlist_ids):
    track_indices = ratings_df[ratings_df['pid'] == playlist_id]['track_id']
    playlist_features[index] = np.mean(additional_features[np.isin(additional_features[:, 0], track_indices)], axis=0)[1:]

# Split data into train and test sets for XGBoost
X_train, X_test, y_train, y_test = train_test_split(playlist_features,
                                                    unique_playlist_ids,
                                                    test_size=0.2, random_state=42)

/usr/local/lib/python3.10/dist-packages/implicit/utils.py:138: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0958864688873291 seconds
  warnings.warn(


  0%|          | 0/50 [00:00<?, ?it/s]

In [23]:
# Convert the playlist IDs to numeric labels for grouping
playlist_id_to_label = {pid: label for label, pid in enumerate(unique_playlist_ids)}
y_train_labels = np.array([playlist_id_to_label[pid] for pid in y_train])
y_test_labels = np.array([playlist_id_to_label[pid] for pid in y_test])

# Calculate the counts of tracks in each playlist
y_train_counts = [np.sum(y_train_labels == label) for label in np.unique(y_train_labels)]

# Initialize and train XGBoost model for ranking
xgb_model = xgb.XGBRanker()
xgb_model.fit(X_train, y_train_labels, group=y_train_counts)

# Get recommendations for a specific playlist
playlist_index = np.where(unique_playlist_ids == 3884)[0][0]
playlist_feature = playlist_features[playlist_index].reshape(1, -1)
recommended_playlist_indices = xgb_model.predict(playlist_feature)

# Convert the float indices to integers
recommended_playlist_indices = recommended_playlist_indices.astype(int)

# Map recommended playlist indices back to playlist IDs
recommended_playlist_ids = unique_playlist_ids[recommended_playlist_indices]


In [25]:
!pip install lightfm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lightfm: filename=lightfm-1.17-cp310-cp310-linux_x86_64.whl size=808332 sha256=138999c0fe8a751feca68a3245d28cdede584ab9a70dd72c00d95b71236321ce
  Stored in directory: /root/.cache/pip/wheels/4f/9b/7e/0b256f2168511d8fa4dae4fae0200fdbd729eb424a912ad636
Successfully built lightfm


In [48]:
from lightfm import LightFM
from lightfm.evaluation import precision_at_k
from implicit.als import AlternatingLeastSquares
from scipy.sparse import coo_matrix
import numpy as np

# Load your data
# ...

# Convert implicit feedback data to a binary sparse matrix indicating song presence
interaction_matrix = coo_matrix((np.ones(len(ratings_df)), (ratings_df['pid'], ratings_df['track_id'])))

# Train WRMF model to obtain latent factors using implicit library
als_model = AlternatingLeastSquares(factors=50, regularization=0.01, iterations=50)
als_model.fit(interaction_matrix)
user_factors = als_model.user_factors
item_factors = als_model.item_factors

# Initialize LightFM model with WARP loss
model = LightFM(loss='warp', no_components=50, learning_rate=0.05)

# Set user and item factors obtained from ALS as embeddings
model.user_embeddings = user_factors
model.item_embeddings = item_factors

# Train the model using WARP loss and latent factors from implicit library
model.fit(interactions=interaction_matrix, epochs=50, num_threads=4)

# Calculate precision at k for evaluation
#test_precision = precision_at_k(model, interactions=interaction_matrix, k=10).mean()




/usr/local/lib/python3.10/dist-packages/implicit/utils.py:138: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.09408283233642578 seconds
  warnings.warn(


  0%|          | 0/50 [00:00<?, ?it/s]

In [50]:
# Get recommendations for a specific playlist
playlist_index = np.where(unique_playlist_ids == 3884)[0][0]

# Create an array of item indices
item_indices = np.arange(interaction_matrix.shape[1])

# Use the playlist index to get a single user index
user_index = np.array([playlist_index] * len(item_indices), dtype=np.int32)

# Get recommendations using the model's predict method
recommended_track_scores = model.predict(user_index, item_indices)

# Sort the recommended_track_scores and get the top N indices
top_n = 10
top_recommended_indices = np.argsort(recommended_track_scores)[::-1][:top_n]

# Use the indices to get the recommended track IDs
recommended_track_ids = top_recommended_indices


In [51]:
recommended_track_ids

array([ 2723, 16484,  8632, 31776, 21419, 29666,  1390, 24504, 33763,
       22893])

In [59]:
from implicit.als import AlternatingLeastSquares
from lightfm import LightFM
from lightfm.data import Dataset
import xgboost as xgb
from sklearn.model_selection import train_test_split
from scipy.sparse import coo_matrix
import numpy as np

# Load your data
# ...

# Convert implicit feedback data to a binary sparse matrix indicating song presence
interaction_matrix = coo_matrix((np.ones(len(ratings_df)), (ratings_df['pid'], ratings_df['track_id'])))

# Train ALS model
als_model = AlternatingLeastSquares(factors=50, regularization=0.01, use_gpu=True, iterations=50)
als_model.fit(interaction_matrix)

# Extract ALS user and item factors
als_user_factors = als_model.user_factors
als_item_factors = als_model.item_factors

# Initialize LightFM model with WARP loss
model = LightFM(loss='warp', no_components=50, learning_rate=0.05)

# Train LightFM model using ALS user and item factors as embeddings
model.fit(interactions=interaction_matrix, epochs=50, num_threads=4)

# Get recommendations for a specific playlist using LightFM
# Get recommendations for a specific playlist
playlist_index = np.where(unique_playlist_ids == 3884)[0][0]

# Create an array of item indices
item_indices = np.arange(interaction_matrix.shape[1])

# Use the playlist index to get a single user index
user_index = np.array([playlist_index] * len(item_indices), dtype=np.int32)

# Get recommendations using the model's predict method
recommended_track_scores = model.predict(user_index, item_indices)

# Sort the recommended_track_scores and get the top N indices
top_n = 10
top_recommended_indices = np.argsort(recommended_track_scores)[::-1][:top_n]

# Use the indices to get the recommended track IDs
recommended_track_ids = top_recommended_indices

# Create a feature matrix for XGBoost using ALS user factors, ALS item factors, and additional features
user_feature_matrix = als_user_factors[playlist_index]
item_feature_matrix = als_item_factors
additional_features = tracks_df[['danceability', 'energy', 'key', 'loudness', 'mode',
                                 'speechiness', 'acousticness', 'instrumentalness',
                                 'liveness', 'valence', 'tempo']].values
unique_playlist_ids = np.unique(ratings_df['pid'])
playlist_features = np.zeros((len(unique_playlist_ids), additional_features.shape[1] - 1))
for index, playlist_id in enumerate(unique_playlist_ids):
    track_indices = ratings_df[ratings_df['pid'] == playlist_id]['track_id']
    playlist_features[index] = np.mean(additional_features[np.isin(additional_features[:, 0], track_indices)], axis=0)[1:]

# Split data into train and test sets for XGBoost
X_train, X_test, y_train, y_test = train_test_split(playlist_features,
                                                    unique_playlist_ids,
                                                    test_size=0.2, random_state=42)

# Convert the playlist IDs to numeric labels for grouping
playlist_id_to_label = {pid: label for label, pid in enumerate(unique_playlist_ids)}
y_train_labels = np.array([playlist_id_to_label[pid] for pid in y_train])
y_test_labels = np.array([playlist_id_to_label[pid] for pid in y_test])

# Calculate the counts of tracks in each playlist
y_train_counts = [np.sum(y_train_labels == label) for label in np.unique(y_train_labels)]

# Initialize and train XGBoost model for ranking
xgb_model = xgb.XGBRanker()
xgb_model.fit(X_train, y_train_labels, group=y_train_counts)

# Assuming you've trained your LightFM model and XGBoost model as shown earlier

# Get recommendations for a specific playlist using LightFM
playlist_index = np.where(unique_playlist_ids == 3884)[0][0]
recommended_track_scores = xgb_model.predict(playlist_feature)

# Sort the recommended_track_scores and get the top N indices
top_n = 10
top_recommended_indices = np.argsort(recommended_track_scores)[::-1][:top_n]

# Use the indices to get the recommended track IDs
recommended_track_indices = top_recommended_indices

# Map recommended track indices back to track IDs
recommended_track_ids = tracks_df['track_id'].iloc[recommended_track_indices]

# Print the top recommended track IDs
print("Top 10 Recommended Track IDs:")
print(recommended_track_ids)


/usr/local/lib/python3.10/dist-packages/implicit/utils.py:138: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.09214353561401367 seconds
  warnings.warn(


  0%|          | 0/50 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:182: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(


ValueError: ignored

In [68]:
# Convert the playlist IDs to numeric labels for grouping
playlist_id_to_label = {pid: label for label, pid in enumerate(unique_playlist_ids)}
y_train_labels = np.array([playlist_id_to_label[pid] for pid in y_train])
y_test_labels = np.array([playlist_id_to_label[pid] for pid in y_test])

# Calculate the counts of tracks in each playlist
y_train_counts = [np.sum(y_train_labels == label) for label in np.unique(y_train_labels)]

# Initialize and train XGBoost model for ranking
xgb_model = xgb.XGBRanker()
xgb_model.fit(X_train, y_train_labels, group=y_train_counts)

XGBRanker(base_score=None, booster=None, callbacks=None, colsample_bylevel=None,
          colsample_bynode=None, colsample_bytree=None,
          early_stopping_rounds=None, enable_categorical=False,
          eval_metric=None, feature_types=None, gamma=None, gpu_id=None,
          grow_policy=None, importance_type=None, interaction_constraints=None,
          learning_rate=None, max_bin=None, max_cat_threshold=None,
          max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
          max_leaves=None, min_child_weight=None, missing=nan,
          monotone_constraints=None, n_estimators=100, n_jobs=None,
          num_parallel_tree=None, predictor=None, random_state=None, ...)

In [73]:
from implicit.als import AlternatingLeastSquares
from lightfm import LightFM
from lightfm.data import Dataset
import xgboost as xgb
from sklearn.model_selection import train_test_split
from scipy.sparse import coo_matrix
import numpy as np

# Load your data
# ...

# Convert implicit feedback data to a binary sparse matrix indicating song presence
interaction_matrix = coo_matrix((np.ones(len(ratings_df)), (ratings_df['pid'], ratings_df['track_id'])))

# Train ALS model
als_model = AlternatingLeastSquares(factors=50, regularization=0.01, use_gpu=True, iterations=50)
als_model.fit(interaction_matrix)

# Extract ALS user and item factors
als_user_factors = als_model.user_factors
als_item_factors = als_model.item_factors

# Initialize LightFM model with WARP loss
model = LightFM(loss='warp', no_components=50, learning_rate=0.05)

# Train LightFM model using ALS user and item factors as embeddings
model.fit(interactions=interaction_matrix, epochs=50, num_threads=4)

# Get recommendations for a specific playlist using LightFM
# Get recommendations for a specific playlist
playlist_index = np.where(unique_playlist_ids == 3884)[0][0]

# Create an array of item indices
item_indices = np.arange(interaction_matrix.shape[1])

# Use the playlist index to get a single user index
user_index = np.array([playlist_index] * len(item_indices), dtype=np.int32)

# Get recommendations using the model's predict method
recommended_track_scores = model.predict(user_index, item_indices)

# Sort the recommended_track_scores and get the top N indices
top_n = 10
top_recommended_indices = np.argsort(recommended_track_scores)[::-1][:top_n]

# Use the indices to get the recommended track IDs
recommended_track_indices = top_recommended_indices

# Map recommended track indices back to track IDs
recommended_track_ids = tracks_df['track_id'].iloc[recommended_track_indices]

# Print the top recommended track IDs
print("Top 10 Recommended Track IDs:")
print(recommended_track_ids)


/usr/local/lib/python3.10/dist-packages/implicit/utils.py:138: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.08990955352783203 seconds
  warnings.warn(


  0%|          | 0/50 [00:00<?, ?it/s]

Top 10 Recommended Track IDs:
2723     25393
31776     1974
8632     16770
16484    24370
21419    19337
22893    14270
34354     3108
11210    13105
1390     33800
29666    13299
Name: track_id, dtype: int64


In [82]:
from implicit.als import AlternatingLeastSquares
from lightfm import LightFM
import xgboost as xgb
from sklearn.model_selection import train_test_split
from scipy.sparse import coo_matrix
import numpy as np
playlist_id_to_label = {pid: label for label, pid in enumerate(unique_playlist_ids)}
y_train_labels = np.array([playlist_id_to_label[pid] for pid in y_train])
y_test_labels = np.array([playlist_id_to_label[pid] for pid in y_test])

# Calculate the counts of tracks in each playlist
y_train_counts = [np.sum(y_train_labels == label) for label in np.unique(y_train_labels)]

# Initialize and train XGBoost model for ranking
xgb_model = xgb.XGBRanker()
xgb_model.fit(X_train, y_train_labels, group=y_train_counts)

playlist_index = np.where(unique_playlist_ids ==3884)[0][0]
recommended_track_scores = xgb_model.predict(playlist_features)

# Sort the recommended_track_scores and get the top N indices
top_n = 10
top_recommended_indices = np.argsort(recommended_track_scores)[::-1][:top_n]

# Use the indices to get the recommended track IDs
recommended_track_indices = top_recommended_indices

# Map recommended track indices back to track IDs
recommended_track_ids = tracks_df['track_id'].iloc[recommended_track_indices]

# Print the top recommended track IDs
print("Top 10 Recommended Track IDs:")
print(recommended_track_ids)


Top 10 Recommended Track IDs:
28999    14872
9684     32757
9658     29225
9659     25230
9660      9221
9661     10521
9662      7141
9663      2598
9664     32296
9665     17457
Name: track_id, dtype: int64
